In [1]:
from flask import Flask, render_template, request, session, url_for, redirect, jsonify
import pymysql
import pandas as pd
app = Flask(__name__)
app.secret_key = 'random string'

#Database Connection
def dbConnection():
    connection = pymysql.connect(host="localhost", user="root", password="root", database="newsclassification")
    return connection


#close DB connection
def dbClose():
    dbConnection().close()
    return


@app.route('/index')
@app.route('/')
def index():
    session['userloc']= request.args.get("location")
    locationis=session['userloc']
    print(locationis)
    return render_template('index.html')


@app.route('/home')
def home():
    if 'user' in session:
        ht_cnt = 0
        toi_cnt = 0
        ie_cnt = 0
        d = {'ht': 0, 'toi': 0, 'ie': 0}
        b = {}
        con = dbConnection()
        cursor = con.cursor()
        result_count = cursor.execute('SELECT * FROM userdetails WHERE mobile = %s',
                                      (session['user']))
        res = cursor.fetchone()
        print(res)
        if result_count > 0:
            print(result_count)
            session['uid'] = res[0]
            ht_cnt = res[6]
            toi_cnt = res[7]
            ie_cnt = res[8]
            d['ht'] = ht_cnt
            d['toi'] = toi_cnt
            d['ie'] = ie_cnt

            print(d)
            a = sorted(d.items(), key=lambda x: x[1], reverse=True)
            b.update(a)
            print(b)
            list = []
            for key in b.keys():
                list.append(key)
            print(list)
            #session['sorted_dict'] = list
        #return render_template('home1.html',user=session['user'], s=session['sorted_dict'])
        return render_template('home.html', user=session['user'], s=list)
    return redirect(url_for('index'))


@app.route('/login', methods=["GET","POST"])
def login():
    msg = ''
    # ht_cnt = 0
    # toi_cnt = 0
    # ie_cnt = 0
    # d = {'ht': 0, 'toi': 0, 'ie': 0}
    # b= {}
    if request.method == "POST":
        # session.pop('user',None)
        mobno = request.form.get("mobile")
        password = request.form.get("pas")
        con = dbConnection()
        cursor = con.cursor()
        result_count = cursor.execute('SELECT * FROM userdetails WHERE mobile = %s AND password = %s',(mobno, password))
        res = cursor.fetchone()
        print(res)
        if result_count > 0:
            print(result_count)
            session['user'] = mobno
            session['uid'] = res[0]
            # ht_cnt = res[6]
            # toi_cnt = res[7]
            # ie_cnt = res[8]
            # d['ht'] = ht_cnt
            # d['toi'] = toi_cnt
            # d['ie'] = ie_cnt
            # print(d)
            # a = sorted(d.items(), key=lambda x: x[1], reverse=True)
            # b.update(a)
            # print(b)
            # list = []
            # for key in b.keys():
            #     list.append(key)
            # print(list)

            # session['sorted_dict']= list
            return redirect(url_for('home'))
        else:
            print(result_count)
            msg = 'Incorrect username/password!'
            return render_template('login.html')
    return render_template('login.html')

@app.route('/register', methods=["GET","POST"])
def register():
    print("register")
    if request.method == "POST":
        try:
            name = request.form.get("name")
            address = request.form.get("address")
            mailid = request.form.get("mailid")
            mobile = request.form.get("mobile")
            pass1 = request.form.get("pass1")
            con = dbConnection()
            cursor = con.cursor()
            cursor.execute('SELECT * FROM userdetails WHERE mobile = %s', (mobile))
            res = cursor.fetchone()
            if not res:
                sql = "INSERT INTO userdetails (name, address, email, mobile, password) VALUES (%s, %s, %s, %s, %s)"
                val = (name, address, mailid, mobile, pass1)
                cursor.execute(sql, val)
                con.commit()
                
                sql1 = "INSERT INTO readingcount (uid, ht_count, toi_count, ie_count) VALUES (%s, %s, %s, %s)"
                val1 = (mobile,int(0),int(0),int(0))
                cursor.execute(sql1, val1)
                con.commit()
                status= "success"
                return redirect(url_for('index'))
            else:
                status = "Already available"
            return status
        except Exception as inst:
            print(inst)
            print("Exception occured at user registration")
            return redirect(url_for('index'))
        finally:
            dbClose()
    return render_template('register.html')


#logout code
@app.route('/news', methods=['GET', 'POST'])
def news():
    if 'user' in session:
        con = dbConnection()
        cursor = con.cursor()
        if 'view' in request.args:
            n = request.args['view']
            if n == 'ie':
                icount = 0
                cursor.execute('SELECT * FROM userdetails WHERE mobile = %s', (session['user']))
                res = cursor.fetchone()
                icount = res[8]
                icount1 = icount + 1
                cursor.execute('update userdetails SET ie_count = %s WHERE mobile = %s', (icount1, session['user']))
                con.commit()
            if n == 'toi':
                icount = 0
                cursor.execute('SELECT * FROM userdetails WHERE mobile = %s', (session['user']))
                res = cursor.fetchone()
                icount = res[7]
                icount1 = icount + 1
                cursor.execute('update userdetails SET toi_count = %s WHERE mobile = %s', (icount1, session['user']))
                con.commit()
            if n == 'ht':
                icount = 0
                cursor.execute('SELECT * FROM userdetails WHERE mobile = %s', (session['user']))
                res = cursor.fetchone()
                icount = res[6]
                icount1 = icount + 1
                cursor.execute('update userdetails SET ht_count = %s WHERE mobile = %s', (icount1, session['user']))
                con.commit()
        if 'location' in request.args:
            session['loc']= request.args['location']
            #session['userloc']= request.args.get("location")
        return redirect(url_for('city'))
        #return render_template('home.html',user=session['user'])
    return redirect(url_for('index'))


#logout code
@app.route('/citynews', methods=['GET', 'POST'])
def citynews():
    if 'user' in session:
        if 'location' in request.args:
            location= request.args['location']
        print("location is "+location)   
        Corpus = pd.read_csv(r"allnews.csv",encoding='latin-1',error_bad_lines=False)
        Corpus=Corpus[Corpus['label']==location+".txt"]
        
        
        
        return render_template("citynews.html",name=location, data=Corpus.to_html())
        #return render_template('home.html',user=session['user'])
    return redirect(url_for('index'))


@app.route('/city')
def city():
    if 'user' in session:
        loc ='mumbai' #session['userloc']#'mumbai'# session['loc']
        print(loc)
        return render_template('city.html', user=session['user'], loc=loc)
    return redirect(url_for('index'))


#logout code
@app.route('/logout')
def logout():
    session.pop('user')
    return redirect(url_for('index'))


if __name__ == '__main__':
    app.run('0.0.0.0')
    #app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [17/May/2020 18:36:12] "GET / HTTP/1.1" 200 -


None


127.0.0.1 - - [17/May/2020 18:36:12] "GET /static/css/bootstrap.css HTTP/1.1" 200 -
127.0.0.1 - - [17/May/2020 18:36:12] "GET /static/css/style.css HTTP/1.1" 200 -
127.0.0.1 - - [17/May/2020 18:36:12] "GET /static/css/font-awesome.min.css%20 HTTP/1.1" 200 -
127.0.0.1 - - [17/May/2020 18:36:12] "GET /static/images/shape2.png HTTP/1.1" 200 -
127.0.0.1 - - [17/May/2020 18:36:12] "GET /static/images/shape1.png HTTP/1.1" 200 -
127.0.0.1 - - [17/May/2020 18:36:12] "GET /static/images/banner.png%20 HTTP/1.1" 200 -
127.0.0.1 - - [17/May/2020 18:36:13] "GET /static/images/shape5.png HTTP/1.1" 200 -
127.0.0.1 - - [17/May/2020 18:36:13] "GET /static/images/shape6.png HTTP/1.1" 200 -
127.0.0.1 - - [17/May/2020 18:36:13] "GET /static/images/shape4.png HTTP/1.1" 200 -
127.0.0.1 - - [17/May/2020 18:36:13] "GET /static/images/shape3.png HTTP/1.1" 200 -
127.0.0.1 - - [17/May/2020 18:36:13] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [17/May/2020 18:36:17] "GET /login HTTP/1.1" 200 -
127.0.0.1 - - [

(1, 'abcd', 'mumbai', 'abc@gmail.com', '7894561230', '123', 8, 8, 6)
1


...

mumbai
location is mumbai


127.0.0.1 - - [17/May/2020 18:37:03] "GET /citynews?location=mumbai HTTP/1.1" 200 -
127.0.0.1 - - [17/May/2020 18:37:10] "GET /citynews?location=pune HTTP/1.1" 200 -


location is pune


127.0.0.1 - - [17/May/2020 18:37:16] "GET /citynews?location=delhi HTTP/1.1" 200 -


location is delhi


In [7]:
location= 'pune'
print(location)   
Corpus = pd.read_csv(r"allnews.csv",encoding='latin-1',error_bad_lines=False)
Corpus=Corpus[Corpus['label']==location+".txt"]

pune


b'Skipping line 23: expected 2 fields, saw 4\n'


In [8]:
Corpus

,text,label
0,PUNE: Pimpri Chinchwad police on Tuesday booke...,pune.txt
1,The violence that ensued in the first two day...,pune.txt
13,"PUNE: Lt Gen Manoj Mukund Naravane, who is set...",pune.txt
14,Pune Municipal Corporation (PMC) has launched ...,pune.txt
15,"The Pune Municipal Corporation (PMC), on Wedne...",pune.txt
16,As many as 22 medical associations IN the city...,pune.txt
19,PUNE: Motorcycle-borne men snatched two gold c...,pune.txt
27,PUNE: Motorcycle-borne men snatched two gold c...,pune.txt
28,"If it werenât for an alert bus driver, commu...",pune.txt
41,Cotton ginners in Maharashtra will soon be ab...,pune.txt
